### Работа с БД из Python

In [2]:
import sqlite3
import pandas as pd

In [3]:
print(sqlite3.version)

2.6.0


Подключение к БД

In [4]:
connection = sqlite3.connect("chinook.db")

Создаем один или несколько курсоров (текущий набор записей)

In [5]:
cursor = connection.cursor()

In [6]:
query = """select "Albumid", "Title", "Artistid" from albums order by title limit 3;"""
albums = cursor.execute(query).fetchall()

In [7]:
albums

[(156, '...And Justice For All', 50),
 (257,
  '20th Century Masters - The Millennium Collection: The Best of Scorpions',
  179),
 (296, 'A Copland Celebration, Vol. I', 230)]

In [8]:
pd.DataFrame(albums)

,0,1,2
0,156,...And Justice For All,50
1,257,20th Century Masters - The Millennium Collecti...,179
2,296,"A Copland Celebration, Vol. I",230


In [9]:
cursor.close()

# connection.commit()
connection.close()

Проще воспользоваться методом pandas - **read_sql_query**.

In [10]:
connection = sqlite3.connect("chinook.db")
cursor = connection.cursor()

In [11]:
dfalbum = pd.read_sql_query(query, connection)
dfalbum

,AlbumId,Title,ArtistId
0,156,...And Justice For All,50
1,257,20th Century Masters - The Millennium Collecti...,179
2,296,"A Copland Celebration, Vol. I",230


In [12]:
cursor.close()

# connection.commit()
connection.close()

### Синтаксис языка SQL

#### CREATE TABLE

`CREATE TABLE [IF NOT EXISTS] 
[schema_name].table_name (
column_1 data_type PRIMARY KEY,
column_2 data_type NOT NULL,
column_3 data_type DEFAULT 0,
	table_constraints
 ) [WITHOUT ROWID]; 
`

In [11]:
connection = sqlite3.connect("chinook.db")
cursor = connection.cursor()


query = """CREATE TABLE managers (
            contact_id INTEGER PRIMARY KEY,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            email TEXT NOT NULL UNIQUE,
            phone TEXT NOT NULL UNIQUE 
        );"""
_ = cursor.execute(query).fetchall()

connection.commit()
connection.close()

Для обеспечения категорной целостности в языке SQL существуют спецификации **PRIMARY KEY** (первичный ключ) и **UNIQUE** (уникальный ключ). 

 - PRIMARY KEY не может быть NULL
 - PRIMARY KEY может быть только один

In [12]:
connection = sqlite3.connect("chinook.db")
query = """SELECT  * FROM managers"""
managers = pd.read_sql_query(query, connection)
connection.close()

In [13]:
managers

,contact_id,first_name,last_name,email,phone


#### INSERT

Вставка новых записей в таблицу.

`
INSERT INTO table1 (column1,column2 ,..)
VALUES 
   (value1,value2 ,...),
   (value1,value2 ,...),
    ...
   (value1,value2 ,...);`

In [14]:
connection = sqlite3.connect("chinook.db")
cursor = connection.cursor()

In [15]:
query = """INSERT INTO managers (first_name, last_name, email, phone)
            VALUES
                ('Buddy', 'Rich', 'buddyrich@gmail.com', '79671950123'),
                -- ('Sam', 'Johnes', 'sam@gmail.com', NULL)
                -- ('Sam', 'Johnes', 'sam@gmail.com', '79671950123')
                ('Sam', 'Johnes', 'sam@gmail.com', '79671950125')
                ;"""
_ = cursor.execute(query).fetchall()

connection.commit()

In [16]:
query = """SELECT  * FROM managers"""
managers = pd.read_sql_query(query, connection)

In [17]:
managers

,contact_id,first_name,last_name,email,phone
0,1,Buddy,Rich,buddyrich@gmail.com,79671950123
1,2,Sam,Johnes,sam@gmail.com,79671950125


Если отсутствует список столбцов, то список вставляемых значений должен быть полный, то есть обеспечивать значения для всех столбцов таблицы. При этом порядок значений должен соответствовать порядку, заданному оператором CREATE TABLE для таблицы, в которую вставляются строки.

#### UPDATE

`
UPDATE table
SET column_1 = new_value_1,
    column_2 = new_value_2
WHERE
    search_condition 
ORDER column_or_expression
LIMIT row_count OFFSET offset;`

In [18]:
query = """UPDATE managers SET first_name = 'Ivan' WHERE phone = '79671950125';"""
_ = cursor.execute(query).fetchall()

connection.commit()

In [19]:
query = """SELECT  * FROM managers"""
managers = pd.read_sql_query(query, connection)

In [20]:
managers

,contact_id,first_name,last_name,email,phone
0,1,Buddy,Rich,buddyrich@gmail.com,79671950123
1,2,Ivan,Johnes,sam@gmail.com,79671950125


#### DELETE

In [21]:
query = """DELETE FROM managers WHERE phone = '79671950123';"""
_ = cursor.execute(query).fetchall()

connection.commit()

In [22]:
query = """SELECT  * FROM managers"""
managers = pd.read_sql_query(query, connection)

In [23]:
managers

,contact_id,first_name,last_name,email,phone
0,2,Ivan,Johnes,sam@gmail.com,79671950125


#### DROP

`
DROP TABLE [IF EXISTS] [schema_name.]table_name;`

In [24]:
query = """DROP TABLE IF EXISTS managers;"""
_ = cursor.execute(query).fetchall()

connection.commit()

In [25]:
connection.close()

#### SELECT

Оператор SELECT осуществляет выборку из базы данных.

In [15]:
connection = sqlite3.connect("chinook.db")
cursor = connection.cursor()

In [17]:
query = """SELECT  * FROM employees"""
df = pd.read_sql_query(query, connection)
df.head(3)

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com


In [18]:
query = """SELECT LastName, FirstName, BirthDate FROM employees"""
df = pd.read_sql_query(query, connection)
df.head(3)

,LastName,FirstName,BirthDate
0,Adams,Andrew,1962-02-18 00:00:00
1,Edwards,Nancy,1958-12-08 00:00:00
2,Peacock,Jane,1973-08-29 00:00:00


#### DISTINCT

Только уникальные значения.

In [19]:
query = """SELECT City FROM employees"""
df = pd.read_sql_query(query, connection)
df.shape

(8, 1)

In [20]:
df

,City
0,Edmonton
1,Calgary
2,Calgary
3,Calgary
4,Calgary
5,Calgary
6,Lethbridge
7,Lethbridge


In [21]:
query = """SELECT DISTINCT City FROM employees"""
df = pd.read_sql_query(query, connection)
df.shape

(3, 1)

In [22]:
df

,City
0,Edmonton
1,Calgary
2,Lethbridge


#### ORDER BY

Cортировка запроса по полю.

In [23]:
query = """SELECT City, BirthDate FROM employees ORDER BY BirthDate"""
df = pd.read_sql_query(query, connection)
df

,City,BirthDate
0,Calgary,1947-09-19 00:00:00
1,Calgary,1958-12-08 00:00:00
2,Edmonton,1962-02-18 00:00:00
3,Calgary,1965-03-03 00:00:00
4,Lethbridge,1968-01-09 00:00:00
5,Lethbridge,1970-05-29 00:00:00
6,Calgary,1973-07-01 00:00:00
7,Calgary,1973-08-29 00:00:00


In [24]:
query = """SELECT City, BirthDate FROM employees ORDER BY 2 DESC"""
df = pd.read_sql_query(query, connection)
df

,City,BirthDate
0,Calgary,1973-08-29 00:00:00
1,Calgary,1973-07-01 00:00:00
2,Lethbridge,1970-05-29 00:00:00
3,Lethbridge,1968-01-09 00:00:00
4,Calgary,1965-03-03 00:00:00
5,Edmonton,1962-02-18 00:00:00
6,Calgary,1958-12-08 00:00:00
7,Calgary,1947-09-19 00:00:00


#### WHERE

Задает условие на горизонтальную выборку.

In [25]:
query = """SELECT * FROM tracks WHERE UnitPrice > 0.99"""
df = pd.read_sql_query(query, connection)
df.head(2)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,2819,Battlestar Galactica: The Story So Far,226,3,18,None,2622250,490750393,1.99
1,2820,Occupation / Precipice,227,3,19,None,5286953,1054423946,1.99


#### AND, OR, NOT

In [26]:
query = """SELECT * FROM tracks WHERE UnitPrice > 0.99 
                                AND (Composer is NOT NULL OR GenreId = 18) 
                                AND NOT Name = 'Occupation / Precipice' """
df = pd.read_sql_query(query, connection)
df.head(2)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,2819,Battlestar Galactica: The Story So Far,226,3,18,None,2622250,490750393,1.99
1,2825,A Measure of Salvation,227,3,18,None,2563938,489715554,1.99


#### BETWEEN, IN 

In [27]:
query = """SELECT * FROM tracks WHERE UnitPrice BETWEEN 0.99 AND 1.5 
                                AND GenreId IN (10, 24)
                                ; """
df = pd.read_sql_query(query, connection)
df.head(2)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,360,Vai-Vai 2001,32,1,10,None,276349,9402241,0.99
1,361,X-9 2001,32,1,10,None,273920,9310370,0.99


#### Подвыборки

In [28]:
query = """ SELECT *
            FROM tracks 
            WHERE GenreId IN (SELECT GenreId 
                                FROM genres
                                WHERE Name = 'Opera') """
df = pd.read_sql_query(query, connection)
df

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,3451,"Die Zauberflöte, K.620: ""Der Hölle Rache Kocht...",317,2,25,Wolfgang Amadeus Mozart,174813,2861468,0.99


#### LIKE

- **_**  вместо любого единичного символа в проверяемом значении;

- **%** заменяет последовательность любых символов (число символов в последовательности может быть от 0 и более) в проверяемом значении.

In [29]:
query = """ SELECT DISTINCT Composer
            FROM tracks 
            WHERE Composer LIKE 'U_' """
df = pd.read_sql_query(query, connection)
df

,Composer
0,U2


In [30]:
query = """ SELECT DISTINCT Composer
            FROM tracks 
            WHERE Composer LIKE 'U%' """
df = pd.read_sql_query(query, connection)
df

,Composer
0,Ulrich
1,U2
2,U2 & Daragh O'Toole
3,U2 & Van Dyke Parks
4,U2; Bono
5,"U2; Edge, The"
6,"U2; Bono & Edge, The"


In [31]:
query = """ SELECT DISTINCT Composer
            FROM tracks 
            WHERE Composer LIKE '%U%' """
df = pd.read_sql_query(query, connection)
df

,Composer
0,"Angus Young, Malcolm Young, Brian Johnson"
1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho..."
2,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D..."
3,"Steven Tyler, Richie Supa"
4,"Steven Tyler, Joe Perry, Mark Hudson"
...,...
339,"Delroy ""Chris"" Cooper, Donovan Jackson, Earl C..."
340,"Astor Campbell, Delroy ""Chris"" Cooper, Donovan..."
341,Henry Purcell
342,Kurt Weill


#### Агрегаты SUM, MIN, MAX, AVG, COUNT

In [32]:
query = """ SELECT COUNT (DISTINCT Composer)
            FROM tracks 
            WHERE Composer LIKE '%U%' """
df = pd.read_sql_query(query, connection)
df

,COUNT (DISTINCT Composer)
0,344


In [33]:
query = """SELECT AVG(Milliseconds) 
            FROM tracks 
                                ; """
df = pd.read_sql_query(query, connection)
df.head(2)

,AVG(Milliseconds)
0,393599.212104


#### GROUP BY/HAVING

**GROUP BY** используется для объединения результатов выборки по одному или нескольким столбцам.

In [34]:
query = """ SELECT *
            FROM tracks 
            WHERE Bytes = (SELECT MAX(Bytes)  
                                FROM tracks) """
df = pd.read_sql_query(query, connection)
df

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,3224,Through a Looking Glass,229,3,21,None,5088838,1059546140,1.99


In [35]:
query = """ SELECT GenreId, AVG(Milliseconds)/60000 as length
            FROM tracks 
            GROUP BY 1
            ORDER BY 2 DESC"""
df = pd.read_sql_query(query, connection)
df

,GenreId,length
0,20,48.529717
1,18,43.759151
2,21,42.921396
3,19,35.750684
4,22,26.421062
5,3,5.162491
6,15,5.049763
7,13,4.957549
8,24,4.897793
9,2,4.862590


Если предложение **WHERE** определяет предикат для фильтрации строк, то предложение **HAVING** применяется после группировки для определения аналогичного предиката, фильтрующего группы по значениям агрегатных функций.

In [36]:
query = """ SELECT GenreId, AVG(Milliseconds)/60000 as length
            FROM tracks 
            GROUP BY 1
            HAVING length > 5
            ORDER BY 2 DESC"""
df = pd.read_sql_query(query, connection)
df

,GenreId,length
0,20,48.529717
1,18,43.759151
2,21,42.921396
3,19,35.750684
4,22,26.421062
5,3,5.162491
6,15,5.049763


#### Выбор из нескольких источников

In [37]:
query = """ SELECT tracks.*,
                   genres.*
            FROM tracks 
            INNER JOIN genres ON tracks.Genreid = genres.Genreid
            WHERE genres.Name = 'Classical'
            """
df = pd.read_sql_query(query, connection)
df

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,GenreId,Name
0,3359,"Symphony No. 3 in E-flat major, Op. 55, ""Eroic...",268,5,24,Ludwig van Beethoven,356426,5817216,0.99,24,Classical
1,3403,Intoitus: Adorate Deum,272,2,24,Anonymous,245317,4123531,0.99,24,Classical
2,3404,"Miserere mei, Deus",273,2,24,Gregorio Allegri,501503,8285941,0.99,24,Classical
3,3405,Canon and Gigue in D Major: I. Canon,274,2,24,Johann Pachelbel,271788,4438393,0.99,24,Classical
4,3406,"Concerto No. 1 in E Major, RV 269 ""Spring"": I....",275,2,24,Antonio Vivaldi,199086,3347810,0.99,24,Classical
...,...,...,...,...,...,...,...,...,...,...,...
69,3498,"Concerto for Violin, Strings and Continuo in G...",342,4,24,Pietro Antonio Locatelli,493573,16454937,0.99,24,Classical
70,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99,24,Classical
71,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99,24,Classical
72,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99,24,Classical


#### Union

In [38]:
query = """ SELECT FirstName, LastName, 'Employee' AS Type
            FROM employees
            UNION
            SELECT FirstName, LastName, 'Customer'
            FROM customers;
            """
df = pd.read_sql_query(query, connection)
df

,FirstName,LastName,Type
0,Aaron,Mitchell,Customer
1,Alexandre,Rocha,Customer
2,Andrew,Adams,Employee
3,Astrid,Gruber,Customer
4,Bjørn,Hansen,Customer
...,...,...,...
62,Steve,Murray,Customer
63,Terhi,Hämäläinen,Customer
64,Tim,Goyer,Customer
65,Victor,Stevens,Customer


Задание
Анализируем продажи интернет-магазина с помощью SQL.
У нас есть доступ к базе данных онлайн-магазина. В этой базе данных
содержатся различные таблицы, в которых хранится информация о
продуктах и сделанных заказах.
Для решения этой задачи нам потребуются данные из двух таблиц:
«orders» (заказы) и «products» (продукты). Все необходимые данные
можно загрузить по следующей
ссылке: https://cloud.mail.ru/public/n4JN/yozGwKhyA
Структура таблиц выглядит следующим образом.Таблица «orders»
(заказы) включает следующие колонки:
- order_id (уникальный номер, который отличает каждый заказ);
- order_date (дата, когда был сделан заказ);
- product_id (идентификационный номер продукта, который был
заказан);
- quantity (количество заказанных единиц каждого продукта);
- total_price (общая стоимость, которую покупатель должен заплатить
за заказ). Таблица «products» (продукты) состоит из следующих
колонок:
- product_id (уникальный номер, который отличает каждый продукт);
- product_name (название каждого продукта);
- category (категория, к которой принадлежит продукт, - например,
книги, электроника и т. д.);
- price (цена каждого продукта).
ВОПРОС:
1. У какого продукта наибольшее количество продаж за весь период?

In [39]:
import sqlite3

# Подключение к базе данных
conn = sqlite3.connect('my_online_store.db')

# Создание курсора
c = conn.cursor()

# SQL-запрос
c.execute("""
SELECT p.product_name, SUM(o.quantity) as total_quantity
FROM orders o
JOIN products p ON o.product_id = p.product_id
GROUP BY p.product_name
ORDER BY total_quantity DESC
LIMIT 1;
""")

# Получение результата
result = c.fetchone()

# Вывод результата
print(f"Продукт с наибольшим количеством продаж: {result[0]}, количество: {result[1]}")

# Закрытие соединения
conn.close()

Продукт с наибольшим количеством продаж: HFvpNNVhmAUgDZ, количество: 21


Чему равна общая выручка с точностью до одного знака после запятой?

In [40]:
import sqlite3

# Подключение к базе данных
conn = sqlite3.connect('my_online_store.db')

# Создание курсора
c = conn.cursor()

# SQL-запрос
c.execute("""
SELECT ROUND(SUM(total_price), 1) as total_revenue
FROM orders;
""")

# Получение результата
result = c.fetchone()

# Вывод результата
print(f"Общая выручка: {result[0]}")

# Закрытие соединения
conn.close()

Общая выручка: 145545.4
